In [1]:
import os
import sys
import copy
pdir = os.path.dirname(os.getcwd())
sys.path.append(pdir)

import torch
import numpy as np

import utils
import wandb

model_dir = 'models'
data_dir = 'tree_points.pt'

In [2]:
trees_data = torch.load(data_dir)

In [3]:
sweep_config = {
  "name":"laser-trees-bayes",
  "method":"bayes",
  "metric":{
      "name":"Best_acc",
      "goal":"maximize"
  },
  "parameters":{
    "loss_fn":{
      "values":["smooth-loss", "cross-entropy"]
    },
    "lr_init":{
      "min": 0.0001,
      "max": 0.001
    },
    "lr_step":{
      "min": 10,
      "max": 100
    },
    "lr_gamma":{
      "min": 0.1,
      "max": 0.9
    },
    "max_rotation":{
      "min":0,
      "max":2*np.pi
    },
    "max_translation":{
      "min":0,
      "max":2.0
    },
    "jitter_std":{
      "distribution":"log_uniform",
      "min":-7.0,
      "max":-2.0
    },
    "min_scale":{
      "min":0.3,
      "max":1
    },
    "max_scale":{
      "min":1,
      "max":2
    }
  }
}

In [4]:
def interface_to_train():
    with wandb.init() as run:
        params={
        "dataset_type":type(trees_data),
        "batch_size":128,
        "validation_split":.15,
        "test_split":.15,
        "shuffle_dataset":True,
        "random_seed":0,
        "learning_rate":[wandb.config["lr_init"], wandb.config["lr_step"], wandb.config["lr_gamma"]],  #[init, step_size, gamma] for scheduler
        "momentum":0.9, #Only used for sgd
        "epochs":300,
        "loss_fn":wandb.config["loss_fn"],
        "optimizer":"adam",
        "voting":"None",
        "train_sampler":"balanced",

        "model":"SimpleView",

        "image_dim":256,
        "camera_fov_deg":90,
        "f":1,
        "camera_dist":1.4,
        "depth_averaging":"min",
        "soft_min_k":50,
        "num_views":6,

        "transforms":['rotation','translation','scaling'], #,'translation'
        "min_rotation":0,
        "max_rotation":wandb.config["max_rotation"],
        "min_translation":0,
        "max_translation":wandb.config["max_translation"],
        "jitter_std":wandb.config["jitter_std"],
        "min_scale":wandb.config["min_scale"],
        "max_scale":wandb.config["max_scale"],

        "species":["QUEFAG", "PINNIG", "QUEILE", "PINSYL", "PINPIN"],
        "data_resolution":"2.5cm"
        }
        
        utils.train(data_dir=data_dir, model_dir=model_dir, params=params, init_wandb=False)
    return

In [5]:
sweep_id = wandb.sweep(sweep_config, project='laser-trees-bayes')

count = 1
wandb.agent(sweep_id, function=interface_to_train, count=count)

Create sweep with ID: d7adac1p
Sweep URL: https://wandb.ai/mja2106/laser-trees-bayes/sweeps/d7adac1p


wandb: Agent Starting Run: e9ixu70o with config:
wandb: 	jitter_std: 0.0010030336971183322
wandb: 	loss_fn: cross-entropy
wandb: 	lr_gamma: 0.4959377105550281
wandb: 	lr_init: 0.0009744788589779739
wandb: 	lr_step: 86
wandb: 	max_rotation: 6.12097546204201
wandb: 	max_scale: 1
wandb: 	max_translation: 1
wandb: 	min_scale: 0.8358648834186979
wandb: Currently logged in as: mja2106 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Config item 'loss_fn' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_rotation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_translation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'jitter_std' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_scale' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_scale' was locked by 'sweep' (ignored update).


QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485
Removing: JUNIPE
Removing: NA
Removing: DEAD
Removing: QUERCUS
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Shuffling dataset...
Using balanced sampling...
Using cross-entropy loss...
Optimizing with AdaM...
Using step LR schedul

/home/matt/anaconda3/envs/laser-trees/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,     5] loss: 3.854
[1,    10] loss: 3.505
[1,    15] loss: 3.549
[1,    20] loss: 3.542
OVERALL: Got 18 / 372 with accuracy 4.84
PINNIG: Got 0/93 with accuracy 0.00
PINPIN: Got 18/18 with accuracy 100.00
PINSYL: Got 0/40 with accuracy 0.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[2,     5] loss: 3.466
[2,    10] loss: 3.404
[2,    15] loss: 3.376
[2,    20] loss: 3.298
OVERALL: Got 43 / 372 with accuracy 11.56
PINNIG: Got 3/93 with accuracy 3.23
PINPIN: Got 0/18 with accuracy 0.00
PINSYL: Got 40/40 with accuracy 100.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[3,     5] loss: 3.227
[3,    10] loss: 3.266


wandb: Ctrl + C detected. Stopping sweep.


PINNIG Accuracy,0.03226
PINPIN Accuracy,0.0
PINSYL Accuracy,1.0
QUEFAG Accuracy,0.0
QUEILE Accuracy,0.0
Best_acc,0.11559
Best_min_acc,0.0
Train Loss,15.59875
Validation Loss,2.95274
Train Accuracy,0.21146
Validation Accuracy,0.11559


PINNIG Accuracy,▁█
PINPIN Accuracy,█▁
PINSYL Accuracy,▁█
QUEFAG Accuracy,▁▁
QUEILE Accuracy,▁▁
Best_acc,▁█
Best_min_acc,▁▁
Train Loss,▁█
Validation Loss,▁█
Train Accuracy,▁█
Validation Accuracy,▁█
